In [1]:
import os
import json
import cv2
import shutil
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
# read json file
viz = False
json_data =  'cropped_data/cropped_annotation/cropped_annotation_mark_added.json' #'annotations/cropped_sampled_seed4.json'
json_object = json.load(open(json_data, "r", encoding='utf-8'))
recon_path = '/home/liverusg/PycharmProjects/AI_USG_liver/codes/Pluralistic-Inpainting/results/usg/'

output_path = 'cropped_data/cropped_images_recon/'

# loop through the region
for k,v in tqdm(json_object.items()):

    # Get dataset type
    data_type = v['set']
    zone = data_type.split("_")[0].strip()
    pos_neg = data_type.split("_")[-1].strip()
    
    regions = v['regions']
    
    # load original image and get shape
    file_path  = v['jpg_img_path']
    ori_img = cv2.imread(file_path)
    ori_height, ori_width, ori_channel = ori_img.shape
    
    found_mark = False
    for l in regions:
        if 'markings' in l['region_attributes'].keys():
            found_mark = True
            break
            
    # if not find mark in region => copy original image to the output folder
    if not found_mark:
        shutil.copy(file_path,output_path+zone+"/"+k)
        
    # else load original and get shape 
    else:
        # load and reshape recon_image follow original image
        recon_img = cv2.imread(recon_path + k[:-4] + "_out_0.png")
        recon_img = cv2.resize(recon_img,(ori_width,ori_height))

        if viz:
            fig = plt.figure(figsize=(20,10))
            fig.suptitle(k[:-4])

            ax1 = fig.add_subplot(1,2,1)
            ax1.imshow(ori_img)
            ax2 = fig.add_subplot(1,2,2)
            ax2.imshow(recon_img)

            plt.show()
        
        for l in regions:
            if 'markings' in l['region_attributes'].keys():
                mbbox = l['bbox']
                # cut and paste mark region from the recon to the origimal image
                ori_img[mbbox['y0']:mbbox['y1'],mbbox['x0']:mbbox['x1']] = recon_img[mbbox['y0']:mbbox['y1'],mbbox['x0']:mbbox['x1']]
                                                                                     
        # save image to output folder
        cv2.imwrite(output_path+zone+"/"+k,ori_img)

100%|██████████| 96956/96956 [11:31<00:00, 140.31it/s]
